In [1]:
from mcts.mcts_sparse import MctsSparse
from simulator.controller import RandomController, DefaultController
from mcts.mcts_utils import ModelWrapper
from external_map_evaluate_utils import speed_optimizer
from external_map_evaluate_utils import speed_optimizer_mcts_heuristic_speed_up
from simulator.externalMapWrapper import ExternalMapWrapper
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Parameter Setting

In [2]:
# physical parameters for two pickers
rear_to_cam_center_meters = 4.22 # meter
cam_view_width_meters = 0.9 # meter
picker_distance_meters = 1.8 # meter
picking_windows_size = 0.9 # meter, center at picker x
time_step = 5 # seconds

# Picking Parameter Setting
pickers_y_init=[1, 4]
pickers_speed_init=[0.6, 0.4] # apple/s
platform_speed_init = 0.021 # m/s
platform_speed_options = [0.08, 0.05, 0.025, 0.01] # m/s

# derived digitized parameter
BLOCK_SIZE = 0.3 # m
picker_distance_pixels = int(picker_distance_meters/BLOCK_SIZE)
sensing_offset = int((rear_to_cam_center_meters-cam_view_width_meters/2)/BLOCK_SIZE)
cam_view_width_pixels = int(cam_view_width_meters/BLOCK_SIZE)
picking_windows_size_half_pixels = int(picking_windows_size/BLOCK_SIZE) - 1 
max_sensing_limit_meters = rear_to_cam_center_meters + 0.5 * cam_view_width_meters
max_picking_limit_meters = picking_windows_size + picker_distance_meters 
max_sensing_limit_pixel =  int(max_sensing_limit_meters/BLOCK_SIZE) + 1
max_picking_limit_pixel = int(max_picking_limit_meters/BLOCK_SIZE)

print("picker_distance_pixels: ", picker_distance_pixels)
print("picking_windows_size_half_pixels: ", picking_windows_size_half_pixels)
print("sensing_offset: ", sensing_offset)
print("cam_view_width_pixels: ", cam_view_width_pixels)
print("max_sensing_limit_meters: ", max_sensing_limit_meters, "m")
print("max_picking_limit_meters: ", max_picking_limit_meters, "m")
print("max_sensing_limit_pixel: ", max_sensing_limit_pixel)
print("max_picking_limit_pixel: ", max_picking_limit_pixel)


picker_distance_pixels:  6
picking_windows_size_half_pixels:  2
sensing_offset:  12
cam_view_width_pixels:  3
max_sensing_limit_meters:  4.67 m
max_picking_limit_meters:  2.7 m
max_sensing_limit_pixel:  16
max_picking_limit_pixel:  9


## Get Fruit Distribution

In [3]:

fruit_dist_path = "./example_fruit_distribution/fruit_dist.map"
external_map = np.loadtxt(fruit_dist_path)
print("external_map shape: ", external_map.shape)
top_fruits = np.sum(external_map[:3,:])
bottom_fruits = np.sum(external_map[3:,:])
print("total fruit number: ", np.sum(external_map))
print('top fruits: ', top_fruits)
print('bottom fruits: ', bottom_fruits)
print('t/b ratio: ', top_fruits/bottom_fruits)


external_map shape:  (6, 190)
total fruit number:  867.0
top fruits:  327.0
bottom fruits:  540.0
t/b ratio:  0.6055555555555555


### Mode 1: Height and speed fixed mode

In [4]:
# Create an environment
env = ExternalMapWrapper(time_step=time_step, 
                        pickers_y_init=pickers_y_init, 
                        pickers_speed_init=pickers_speed_init,  
                        platform_speed_init=platform_speed_init,
                        rear_to_cam_center_meters=rear_to_cam_center_meters, 
                        cam_view_width_meters=cam_view_width_meters, 
                        picker_distance_meters=picker_distance_meters, 
                        picking_windows_size=picking_windows_size,
                        external_map=external_map)
num_apple_start = env.count_apple_in_valid_range()
print("num_apple_start: ", num_apple_start)


num_apple_start:  852.0


In [5]:
# Run 
total_steps = 0
plot_save_folder = "./results/mode1/"
render = False

while True:
    if render:
        env.appEnv.show_map(save=plot_save_folder)
    action = [0, 0] # [0, 0] means not moving the picker height which is mode 1
    done = env.step(action)
    total_steps += 1
    if done:
        break
num_apple_left = env.count_apple_in_valid_range()
picked_percent = (1 - num_apple_left/num_apple_start)
picked_number = (num_apple_start - num_apple_left)

print("------ Mode 1: Height and speed fixed mode ----")
print("Apple num: ", num_apple_start)
print("Left apple: ", num_apple_left)
print("Picking Percentage: %.2f %%" %(picked_percent*100))
print("Time to Finish %i (s) " %(total_steps*time_step))



------ Mode 1: Height and speed fixed mode ----
Apple num:  852.0
Left apple:  68.0
Picking Percentage: 92.02 %
Time to Finish 2505 (s) 


### Mode 2: Speed-optimized mode

In [6]:
# Create an environment
env = ExternalMapWrapper(time_step=time_step, 
                        pickers_y_init=pickers_y_init, 
                        pickers_speed_init=pickers_speed_init,  
                        platform_speed_init=platform_speed_init,
                        rear_to_cam_center_meters=rear_to_cam_center_meters, 
                        cam_view_width_meters=cam_view_width_meters, 
                        picker_distance_meters=picker_distance_meters, 
                        picking_windows_size=picking_windows_size,
                        external_map=external_map)
num_apple_start = env.count_apple_in_valid_range()
print("num_apple_start: ", num_apple_start)


num_apple_start:  852.0


In [7]:
policy_fn = DefaultController # Height control mode set as fixed
min_picking_percentage = 0.9 # only valid for speed optimized modes (mode 2,4), no use for fixed speed modes (mode 1,3)

policy = policy_fn(env.appEnv)
speed_opt_every_step = True
speed_planing_valid_count = np.inf
valid_planning_timestep = 0
speed_profile_mode2 = []
render = False
total_steps = 0
while True:
    if render:
        env.appEnv.show_map(save="./results/mode2/")
        
    # ----------- Speed optimization ----------------
    if (speed_planing_valid_count > valid_planning_timestep) or speed_opt_every_step:

        opt_speed = speed_optimizer(platform_speed_options, policy_fn, env, pickers_y_init, pickers_speed_init,
                                    time_step=time_step,
                                    rear_to_cam_center_meters=rear_to_cam_center_meters,
                                    cam_view_width_meters=cam_view_width_meters, 
                                    picker_distance_meters=picker_distance_meters,
                                    picking_windows_size=picking_windows_size, 
                                    min_picking_percentage=min_picking_percentage)

        # Set speed
        env.platform_speed = opt_speed
        env.appEnv.platform_speed = opt_speed

        # Calculate speed valid time
        valid_planning_range = env.appEnv.map.shape[1] - env.max_sensing_limit_pixel - (env.appEnv.PICKER_DISTANCE + 2*env.appEnv.PICKING_WINDOWS_SIZE)
        valid_planning_period = (valid_planning_range*env.block_size)/env.platform_speed 
        valid_planning_timestep = min(5, int((valid_planning_period/time_step)/2)) # divide by 2 to increase speed planning frequency in order to deal with sensing changing        
        speed_planing_valid_count = 0

        if not speed_opt_every_step:
            print("planning speed: ", opt_speed)
            print("planning speed valid for : ", valid_planning_timestep, " step") 

    speed_profile_mode2.append(env.appEnv.platform_speed)

    # -----------------------------------------------         
    action = policy.get_action(env.state)
    done = env.step(action)
    total_steps += 1
    if done:
        break

num_apple_left = env.count_apple_in_valid_range()
picked_percent = (1 - num_apple_left/num_apple_start)
picked_number = (num_apple_start - num_apple_left)

print("------ Mode 2: Speed-optimized mode ----")
print("Apple num: ", num_apple_start)
print("Left apple: ", num_apple_left)
print("Picking Percentage: %.2f %%" %(picked_percent*100))
print("Time to Finish %i (s) " %(total_steps*time_step))


------ Mode 2: Speed-optimized mode ----
Apple num:  852.0
Left apple:  36.0
Picking Percentage: 95.77 %
Time to Finish 2880 (s) 


### Mode 3: Height-optimized mode

In [8]:
# Create an environment
env = ExternalMapWrapper(time_step=time_step, 
                        pickers_y_init=pickers_y_init, 
                        pickers_speed_init=pickers_speed_init,  
                        platform_speed_init=platform_speed_init,
                        rear_to_cam_center_meters=rear_to_cam_center_meters, 
                        cam_view_width_meters=cam_view_width_meters, 
                        picker_distance_meters=picker_distance_meters, 
                        picking_windows_size=picking_windows_size,
                        external_map=external_map)
num_apple_start = env.count_apple_in_valid_range()
print("num_apple_start: ", num_apple_start)


num_apple_start:  852.0


In [9]:
plan_depth = 2
sample_num_per_action = 2
extra_search_depth = 5
extra_search_sample = 3
gamma = 0.99


In [10]:
random_policy_fn = RandomController(env.appEnv)
model_fn = ModelWrapper(env.appEnv)

# establish a sparse sampling picker heighet optimizer
sparse_sampling = MctsSparse(env.appEnv, plan_depth, gamma, model_fn, random_policy_fn, 
                                extra_search_depth=extra_search_depth, extra_search_sample=extra_search_sample)
num_apple_start = env.count_apple_in_valid_range()
render = False
total_steps = 0
while True:
    if render:
        env.appEnv.show_map(save="./results/mode3/")
    # Get the optimal picker height control action from the optimizer
    action, best_q = sparse_sampling.run(sample_num_per_action=sample_num_per_action) 
    done = env.step(action)
    total_steps += 1
    if done:
        break

num_apple_left = env.count_apple_in_valid_range()
if num_apple_start!=0:
    picked_percent = (1 - num_apple_left/num_apple_start)
else:
    picked_percent = 1
picked_number = (num_apple_start - num_apple_left)

print("------ Mode 3: Height-optimized mode ----")
print("Apple num: ", num_apple_start)
print("Left apple: ", num_apple_left)
print("Picking Percentage: %.2f %%" %(picked_percent*100))
print("Time to Finish %i (s) " %(total_steps*time_step))


------ Mode 3: Height-optimized mode ----
Apple num:  852.0
Left apple:  31.0
Picking Percentage: 96.36 %
Time to Finish 2505 (s) 


### Mode 4: Full co-robotic mode

In [11]:
# Create an environment
env = ExternalMapWrapper(time_step=time_step, 
                        pickers_y_init=pickers_y_init, 
                        pickers_speed_init=pickers_speed_init,  
                        platform_speed_init=platform_speed_init,
                        rear_to_cam_center_meters=rear_to_cam_center_meters, 
                        cam_view_width_meters=cam_view_width_meters, 
                        picker_distance_meters=picker_distance_meters, 
                        picking_windows_size=picking_windows_size,
                        external_map=external_map)
num_apple_start = env.count_apple_in_valid_range()
print("num_apple_start: ", num_apple_start)


num_apple_start:  852.0


In [12]:
heuristic_policy_fn = RandomController
speed_opt_every_step = True
random_policy_fn = RandomController(env.appEnv)


In [13]:
model_fn = ModelWrapper(env.appEnv)
sparse_sampling = MctsSparse(env.appEnv, plan_depth, gamma, model_fn, random_policy_fn, 
                            extra_search_depth=extra_search_depth, extra_search_sample=extra_search_sample)



total_steps = 0
speed_planing_valid_count = np.inf
valid_planning_timestep = 0
speed_profile_mode4 = []
render = False
while True:
    speed_planing_valid_count += 1
    if render:
        env.appEnv.show_map(save="./results/mode4/")
    
    # ----------- Speed optimization ----------------
    if speed_planing_valid_count > valid_planning_timestep or speed_opt_every_step:
        opt_speed = speed_optimizer_mcts_heuristic_speed_up(platform_speed_options,heuristic_policy_fn, env, pickers_y_init, pickers_speed_init,
                                        plan_depth, sample_num_per_action, extra_search_depth, extra_search_sample, 
                                        time_step=time_step,
                                        rear_to_cam_center_meters=rear_to_cam_center_meters,
                                        cam_view_width_meters=cam_view_width_meters, 
                                        picker_distance_meters=picker_distance_meters,
                                        picking_windows_size=picking_windows_size, 
                                        min_picking_percentage=min_picking_percentage,
                                        verbose=False)            

        # Set speed
        env.platform_speed = opt_speed
        env.appEnv.platform_speed = opt_speed

        # Calculate speed valid time
        valid_planning_range = env.appEnv.map.shape[1] - env.max_sensing_limit_pixel - (env.appEnv.PICKER_DISTANCE + 2*env.appEnv.PICKING_WINDOWS_SIZE)
        valid_planning_period = (valid_planning_range*env.block_size)/env.platform_speed 
        valid_planning_timestep = min(5, int((valid_planning_period/time_step)/2)) # divide by 2 to increase speed planning frequency in order to deal with sensing changing        
        speed_planing_valid_count = 0

        if not speed_opt_every_step:
            print("planning speed: ", opt_speed)
            print("planning speed valid for : ", valid_planning_timestep, " step")

    speed_profile_mode4.append(env.appEnv.platform_speed)
    # ----------- Height optimization ----------------
    action, best_q = sparse_sampling.run(sample_num_per_action=sample_num_per_action)
    done = env.step(action)
    total_steps += 1
    if done:
        break

num_apple_left = env.count_apple_in_valid_range()
picked_percent = (1 - num_apple_left/num_apple_start)
picked_number = (num_apple_start - num_apple_left)

max_picking = env.max_picking()
effective_picking_efficiency = (num_apple_start - num_apple_left)/max_picking


print("------ Mode 4: Co Robotic mode ----")
print("Apple num: ", num_apple_start)
print("Left apple: ", num_apple_left)
print("Picking Percentage: %.2f %%" %(picked_percent*100))
print("Time to Finish %i (s) " %(total_steps*time_step))


------ Mode 4: Co Robotic mode ----
Apple num:  852.0
Left apple:  29.0
Picking Percentage: 96.60 %
Time to Finish 1860 (s) 
